In [194]:
from get_data import scrape_companies_info
import sqlite3

connection = sqlite3.connect("../data/financedata.db")
cursor = connection.cursor()

In [195]:
type(connection)

sqlite3.Connection

In [177]:
def tablefilter(connection, table_name:str, filter:dict, columns:list=None, c="AND"):
    """
    Search data in table  by filter and return the result as sql3 cursor.
    
    :param:
        connection: SQL3 connection object
        table_name: name of table to search
        filter: The filter (in dict) to search for
        columns: List of columns to return (default is all columns)
        c: condition AND, OR
    :return: Dictionary containing company info (return list if have many result) if found, otherwise None
    """
    cursor = connection.cursor()
    
    filter = list(filter.items())
    condition = []
    for q, v in filter:
        if hasattr(v, '__iter__') and not isinstance(v, str):
            condition.append(f"{q} IN {str(tuple(v))}")
        else:
            condition.append("{} = '{}'".format(q, v))
            
    condition = " WHERE " + f" {c} ".join(condition)    

    if columns is None:
        res = cursor.execute(
            f"""
            SELECT *
            FROM {table_name} 
            """ + condition
            )
    else:
        res = cursor.execute(
            f"""
            SELECT {','.join(columns)} 
            FROM {table_name} 
            """ + condition
            )
    return res

In [193]:

def company_info_get(connection, filter:dict, columns:list=None, get_first:bool=True):
    """
    Search for company info by filter and return the result as a dictionary.
    
    :param:
        session: SQLAlchemy session object
        filter: The filter (in dict) to search for
        columns: List of columns to return (default is all columns)
        get_first: If True, return only first object
    :return: Dictionary containing company info (return list if have many result) if found, otherwise None
    """
    cursor = connection.cursor()
    
    res = tablefilter(connection, "companyInfo", filter, columns)
    
    if get_first: return dict(zip(columns, res.fetchone()))

    else: return [ dict(zip(columns, r)) for r in res.fetchall() ]
    

company_info_get(connection, {'cik': 789019,},  ['name', 'ticker', 'cik'], False)

[{'cik': 789019}]

In [179]:
_filter = {
    'cik': [789019, 1652044],
    'ticker': ['AMZN', 'NVDA']
}
columns = ['name', 'ticker', 'cik']

res = tablefilter(connection, "companyInfo", _filter, columns, "OR")

res.fetchall()

[('MICROSOFT CORP', 'MSFT', 789019),
 ('NVIDIA CORP', 'NVDA', 1045810),
 ('Alphabet Inc.', 'GOOGL', 1652044),
 ('AMAZON COM INC', 'AMZN', 1018724),
 ('Alphabet Inc.', 'GOOG', 1652044)]

In [130]:
type(res)

sqlite3.Cursor

In [119]:
condition = []
for q, v in _filter:
    if hasattr(v, '__iter__') and not isinstance(v, str):
        condition.append(f"{q} IN {str(tuple(v))}")
    else:
        condition.append("{} = '{}'".format(q, v))
        
condition = " WHERE " + " OR ".join(condition)    
    
condition

" WHERE cik IN (789019, 1652044) AND ticker = 'AMZN'"

In [105]:
f" IN {str(tuple([1,2]))}"

' IN (1, 2)'

In [65]:
" AND ".join(["{}='{}'".format(*f) for f in _filter])

"cik='1652044' AND ticker='GOOG'"

In [60]:
_filter

[('cik', 1652044), ('ticker', 'GOOG')]

In [44]:
f"SELECT {','.join(columns)}  FROM companyInfo WHERE {list(_filter.keys())[0]}={list(_filter.values())[0]}"

'SELECT name,ticker,cik  FROM companyInfo WHERE ticker=GOOG'

In [29]:
"".join(list(_filter.keys())[0])

'ticker'